# NSIDC - SMAP Data Products

Esta Notebook tiene como finalidad orientas sobre como obtener el conjunto de valores posibles correspondiente a las variables a completar en el script **parameters.py** por parte del usuario al momento de utilizar la funcionalidad para descarga de imagenes correspondientes a la mision **Soil Moisture Active Passive Data (SMAP)**

El uso para la obtencion de caracteristicas correspondiente a otras misiones tiene caracteristicas similares considerando los ajuntes necesarios segun la configuracion requerida para el acceso a los productos para los cuales EARTHDATA disponibiliza informacion.

### 1. Abreviaturas:

* NSIDC: National Snow and Ice Data Center
* DAAC: Distributed Active Archive Center
* EOS: Earth Observing System
* CRM: Common Metadata Repository
* API: Application programming interface
* SMAP: Soil Moisture Active Passive Data

### 2. Documentacion de Referencia

* NSIDC DAAC's data access and service API https://nsidc.org/data/user-resources/help-center/programmatic-data-access-guide#anchor-3

* NSIDC-Data-Access-Notebook https://github.com/nsidc/NSIDC-Data-Access-Notebook

* Table of Key-Value-Pair (KVP) Operands for Subsetting, Reformatting, and Reprojection Services https://nsidc.org/data/user-resources/help-center/table-key-value-pair-kvp-operands-subsetting-reformatting-and-reprojection-services

* Earthdata CRM Search - API Documentation https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#_ga=2.224475802.1037008902.1697547140-1448504569.1697547140

### 3. Credenciales

Para acceder a los datos del NSIDC se requiere una cuenta de inicio de sesión de Earthdata. En caso de no disponer de una cuenta disponible visite http://urs.earthdata.nasa.gov para registrarse.

Luego de registrado en Earthdata es necesario completar los datos de registro en las variables API_USER (usuario) / API_SECRET (contraseña) / EMAIL (email) en las siguientes variables:

    - API_USER = ''
    - API_SECRET = ''
    - API_EMAIL = ''

En caso de trabajar con repositorios y versionadores de codigo queda a responsabilidad del usuario el manejo de los secretos para evitar su publicacion.

In [24]:
API_USER=''
API_SECRET=''
API_EMAIL=''

### 4. SMAP - Pares Clave-Valor (Key-Value-Pair) para subconjunto, reformateo y reproyección.

Se considera como documentacion de referencia https://nsidc.org/data/user-resources/help-center/table-key-value-pair-kvp-operands-subsetting-reformatting-and-reprojection-services 

In [25]:
import requests
import json

### 4.1 Variable 'short_name'

Los posibles valores depende de la mision que se este considerando. Para las misiones que se encuentran disponibles para consulta via API se puede observan los valores por mision en https://nsidc.org/data/user-resources/help-center/table-key-value-pair-kvp-operands-subsetting-reformatting-and-reprojection-services

En particular para SMAP se observan en https://nsidc.org/data/user-resources/help-center/what-data-subsetting-reformatting-and-reprojection-services-are-available-smap-data. A continuacion se toma como ejemplo el valor **SPL3SMP_E**


In [26]:
short_name = 'SPL3SMP_E'

Teniendo identificado el valor de 'short_name' es posible obtener en formato JSON la metadata correspondiente a Common Metadata Repository (CMR)

In [27]:
params = {'short_name': short_name}
cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
response = requests.get(cmr_collections_url, params=params)
results = json.loads(response.content)
results

{'feed': {'updated': '2023-10-17T18:55:40.880Z',
  'id': 'https://cmr.earthdata.nasa.gov:443/search/collections.json?short_name=SPL3SMP_E',
  'title': 'ECHO dataset metadata',
  'entry': [{'processing_level_id': 'Level 3',
    'cloud_hosted': False,
    'boxes': ['-85.044 -180 90 180'],
    'time_start': '2015-03-31T00:00:00.000Z',
    'version_id': '005',
    'dataset_id': 'SMAP Enhanced L3 Radiometer Global and Polar Grid Daily 9 km EASE-Grid Soil Moisture V005',
    'has_spatial_subsetting': True,
    'has_transforms': True,
    'associations': {'services': ['S1568899363-NSIDC_ECS',
      'S2011758375-NSIDC_ECS',
      'S1956579962-NSIDC_ECS',
      'S1977894169-NSIDC_ECS'],
     'tools': ['TL2013523138-NSIDC_ECS',
      'TL2011654705-NSIDC_ECS',
      'TL1977971361-NSIDC_ECS',
      'TL1952642907-NSIDC_ECS',
      'TL2000645101-NSIDC_ECS',
      'TL1979078682-NSIDC_ECS',
      'TL1995500651-NSIDC_ECS',
      'TL1956550964-NSIDC_ECS',
      'TL2012682515-NSIDC_ECS']},
    'has_varia

### 4.2 Variable 'version'

In [28]:
#Se listan todos los valores de 'version_id'

versions = [el['version_id'] for el in results['feed']['entry']]
versions

['005']

In [29]:
#Se muestra la utlima version disponible

latest_version = max(versions)
latest_version

'005'

### 4.3 Variables 'formato', 'coverages' y 'projection'

In [31]:
# Query service capability URL 

from xml.etree import ElementTree as ET

capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{short_name}.{latest_version}.xml'
session = requests.session()
s = session.get(capability_url)
response = session.get(s.url,auth=(API_USER,API_SECRET))
root = ET.fromstring(response.content)

subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]
if len(subagent) > 0 :

    #Coverages
    coverages = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
    coverages_raw = [coverages[i]['value'] for i in range(len(coverages))]
    coverages_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in coverages_raw] 
    coverages_values = [v.replace(':', '/') for v in coverages_join]

    #Formatos
    formats = [Format.attrib for Format in root.iter('Format')]
    format_values = [formats[i]['value'] for i in range(len(formats))]
    format_values.remove('')

    #Proyecciones
    projections = [Projection.attrib for Projection in root.iter('Projection')]

In [32]:
#Formatos disponibles --> Variable 'formato'

format_values

['HDF-EOS5', 'NetCDF4-CF', 'ASCII', 'KML', 'GeoTIFF']

In [33]:
# Coberturas disponibles --> Variable 'coverages'

coverages_values

['/Soil_Moisture_Retrieval_Data_AM',
 '/Soil_Moisture_Retrieval_Data_AM/albedo',
 '/Soil_Moisture_Retrieval_Data_AM/albedo_dca',
 '/Soil_Moisture_Retrieval_Data_AM/albedo_scah',
 '/Soil_Moisture_Retrieval_Data_AM/albedo_scav',
 '/Soil_Moisture_Retrieval_Data_AM/boresight_incidence',
 '/Soil_Moisture_Retrieval_Data_AM/bulk_density',
 '/Soil_Moisture_Retrieval_Data_AM/clay_fraction',
 '/Soil_Moisture_Retrieval_Data_AM/EASE_column_index',
 '/Soil_Moisture_Retrieval_Data_AM/EASE_row_index',
 '/Soil_Moisture_Retrieval_Data_AM/freeze_thaw_fraction',
 '/Soil_Moisture_Retrieval_Data_AM/grid_surface_status',
 '/Soil_Moisture_Retrieval_Data_AM/latitude',
 '/Soil_Moisture_Retrieval_Data_AM/latitude_centroid',
 '/Soil_Moisture_Retrieval_Data_AM/longitude',
 '/Soil_Moisture_Retrieval_Data_AM/longitude_centroid',
 '/Soil_Moisture_Retrieval_Data_AM/radar_water_body_fraction',
 '/Soil_Moisture_Retrieval_Data_AM/retrieval_qual_flag',
 '/Soil_Moisture_Retrieval_Data_AM/retrieval_qual_flag_dca',
 '/Soil_

In [34]:
#Proyecciones disponibles --> Variable 'projection'

projections_values = [projections[i]['label'] for i in range(len(projections))]
projections_values

['Geographic',
 'Universal Transverse Mercator',
 'Cylindrical Equal Area',
 'No Change',
 'North Polar Stereographic',
 'South Polar Stereographic']